<a href="https://colab.research.google.com/github/mchablani/generative_ai/blob/main/pnyx/Pnyx_message_classifier_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Read the CSV file from Google Drive
filename = '/content/drive/My Drive/pnyx/data/messages.csv'
df = pd.read_csv(filename)

# Print the DataFrame
print(df["message"].head(10))


0                   what u think all the guns is foooo
1                    u need to get your prios straight
2                      I got normies saying butti only
3                         Should we get on retail wow?
4                         Apparently 3s is really good
5    And you know I’m not toxic at all or anything ...
6                    <a:goldenkek:1175176571812839465>
7                                       absolutely not
8                                         it aint good
9                          What are u playing rn? Sod?
Name: message, dtype: object


In [ ]:
# Ensure all messages are strings
df['message'] = df['message'].astype(str)

In [ ]:
from transformers import pipeline, BertTokenizer, BertForSequenceClassification
from concurrent.futures import ThreadPoolExecutor


In [ ]:
classifier = pipeline("text-classification", model="mrm8488/bert-tiny-finetuned-sms-spam-detection", device=0)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
tokenizer = BertTokenizer.from_pretrained("mrm8488/bert-tiny-finetuned-sms-spam-detection")


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
model = BertForSequenceClassification.from_pretrained("mrm8488/bert-tiny-finetuned-sms-spam-detection")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [ ]:
import threading
import torch

counter_lock = threading.Lock()
counter = 0

# Prediction function
def predict_message(message):
    global counter

    try:
      inputs = tokenizer.encode(message, return_tensors="pt", max_length=512, truncation=True)
      # Move tensors to GPU if available
      inputs = inputs.to('cuda')
      with torch.no_grad():
          outputs = model(inputs)
      # Get prediction scores
      logits = outputs.logits

      # Convert logits to probabilities and labels
      probs = torch.softmax(logits, dim=1)
      predictions = torch.argmax(probs, dim=1)
      # Decode prediction
      label = classifier.model.config.id2label[predictions.item()]

      # result = classifier(message)
      # return "Spam" if result[0]['label'] == 'LABEL_1' else "Not Spam"
      return "Spam" if label == 'LABEL_1' else "Not Spam"

    except Exception as e:
        print(e)
        with counter_lock:
            counter += 1
        print("=============")
        print(message)
        print("=============")
        return "Spam"

In [ ]:
print(predict_message("And you know I’m not toxic at all or anything ..."))

Not Spam


In [ ]:
print(type(df['message'].tolist()))

<class 'list'>


In [ ]:
message_list = df['message'].tolist()

In [ ]:
# Function to apply predictions using multithreading
def apply_predictions_concurrently(data, function, num_workers=5):
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = list(executor.map(function, data))
    return results

In [ ]:
# Apply prediction using multithreading
df['Prediction'] = apply_predictions_concurrently(df['message'], predict_message, num_workers=10)

In [ ]:
# Split data
spam_df = df[df['Prediction'] == 'Spam']
not_spam_df = df[df['Prediction'] == 'Not Spam']

In [ ]:
# Save to CSV
spam_df.to_csv('/content/drive/My Drive/pnyx/data/spam_messages.csv', index=False)
not_spam_df.to_csv('/content/drive/My Drive/pnyx/data/not_spam_messages.csv', index=False)

In [ ]:
print(counter)

0


In [ ]:
print(len(df))

144390


In [ ]:
print(len(spam_df))

13466


In [ ]:
print(len(not_spam_df))

130924
